<a href="https://colab.research.google.com/github/kasivisu4/NEU_CLASS/blob/main/Algorithms_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install anytree

     |████████████████████████████████| 41 kB 550 kB/s 


In [ ]:
import csv
from anytree import Node, RenderTree
import sys

In [ ]:

root=Node("root")
nodes={}
nodes[root.name]=root

In [ ]:

# we create the key representing the start char of every word in the sentence like if the sentence is like "can we create the key and find similar keys?" in the alphabet sequence array
# The advantage of this approach is that we can find the similar keys in short time
# The first_char_array_title will contains [c,w,t,k,a,f,s] unique values
# The key will be 10100100001000000011001000 because we assign 
# 
# for Char 'c' => arr[2]=1 ; Char 'w' => arr[22]=1 ; Char 't' => arr[19]=1 ; Char 'k' => arr[10]=1 ; Char 'a' => arr[0]=1 ; Char 'f' => arr[5]=1 ; Char 's' => arr[18]=1 ; 

def create_node_key(title):

    # char_array = ['0']*26
    first_char_array_title =set(map(lambda x: x[0].lower() if x[0].lower()<='z' and x[0].lower()>='a' else "" ,filter(lambda x:x!="",title.strip().split(' '))))
    char_array = first_char_array_title
    # for start_char in first_char_array_title:
    #     index= ord(start_char.lower()) - ord('a')
    #     if  index >=0 and index<=25:
    #         char_array[index] = '1'

    key = "".join(char_array)
    return key

# # We are finding the similar keys by making the key into binary format and simply using the bitwise and operator to see if the values !=0 which means the key1 and key2 are having similar keys
# def cmp_key(key1,key2):
#     return (eval('0b' + key1) & eval('0b' + key2) !=0)

In [ ]:

tree_pointer = {}
tree_pointer ["Segment Title"]={}
tree_pointer ["Family Title"]={}
tree_pointer ["Class Title"]={}
tree_pointer ["Commodity Title"]={}


def create_Node(data,title):
    try:
      global tree_pointer
      parent_key=""
      if (data["key"].strip()==''):
          return

      node_key = create_node_key(data['title'])
      if node_key in tree_pointer[title]:
        parent_key = tree_pointer[title][node_key]
    
      else:
         tree_pointer[title][node_key] = Node(node_key,parent=nodes[title])
         parent_key =  tree_pointer[title][node_key]

      if not data["title"] in set(map(lambda child: child.name,parent_key.children)):   
          Node(data["title"],parent=parent_key,data=data)

    except:
      print(sys.exc_info()[2])
      print(data)

Data Extraction from the UNSPSC File and storing the data in the tree structure:

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/UNSPSC_English.csv',encoding="utf8") as unspsc_file:
    [ next(unspsc_file) for _ in range(9)]   
    unspsc_reader = csv.DictReader(unspsc_file, delimiter=',')
    [ next(unspsc_reader) for _ in range(2)] 
    titles=[["Segment Title","Segment"],["Family Title","Family"],["Class Title","Class"],["Commodity Title","Commodity"]]
    for title in titles:
      nodes[title[0]] = Node(title[0],parent=root)

    for line in unspsc_reader:
      for index in range(0,len(titles)):
        create_Node({"title" : line[titles[index][0]],"key":line[titles[index][1]]},titles[index][0])

          


**Unique Values of titles in UNSPSC File**

---



In [ ]:

print("Segement Title Leaves Count: ",len(nodes["Segment Title"].leaves),"                 Segement Title Children Count: ",len(nodes["Segment Title"].children))

print("Family Title Leaves Count: ",len(nodes["Family Title"].leaves),"                  Family Title Children Count: ",len(nodes["Family Title"].children))

print("Class Title Leaves Count: ",len(nodes["Class Title"].leaves),"                  Class Title Children Count: ",len(nodes["Class Title"].children))

print("Commodity Title Leaves Count: ",len(nodes["Commodity Title"].leaves),"            Commodity Title Children Count: ",len(nodes["Commodity Title"].children))

Segement Title Leaves Count:  57                  Segement Title Children Count:  57
Family Title Leaves Count:  546                   Family Title Children Count:  425
Class Title Leaves Count:  7902                   Class Title Children Count:  3328
Commodity Title Leaves Count:  147893             Commodity Title Children Count:  30797


In [ ]:
from anytree.exporter import DotExporter
DotExporter(nodes["Segment Title"]).to_picture("Segment_title.png")

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Avalara_goods_and_services.xlsx', 'goods_and_services', header=1)
data_array = np.array(df)
avalara_file = data_array[1706:2547]

In [ ]:
create_node_key_vf = np.vectorize(create_node_key)
avalara_file = np.column_stack((avalara_file, create_node_key_vf(avalara_file[:,1])))

In [ ]:
memoization = {}
memoization["Commodity Title"] = {}
memoization["Class Title"] = {}
memoization["Family Title"] = {}
memoization["Segment Title"] = {}

def store_leave_addr():
    for level in tree_pointer:
        for node in tree_pointer[level]:
            for i in node:
                if i in memoization[level]:
                    for k in tree_pointer[level][node].children:
                      memoization[level][i].append(k)
                else:
                    memoization[level][i] = []
                    for k in tree_pointer[level][node].children:
                      memoization[level][i].append(k)

In [ ]:
store_leave_addr()

In [ ]:
def search_tree(key,level):
    key_match = []
    global memoization
    # key_match["Commodity Title"] = {}
    # key_match["Class Title"] = {}
    # key_match["Family Title"] = {}
    # key_match["Segment Title"] = {}
    for i in key:
        if i in memoization[level]:
          for j in memoization[level][i]:
              key_match.append(j)
 
    return key_match

In [ ]:
def longestCommonSubsequence(text1, text2):

    text1 = text1.lower().split(" ")
    text2 = text2.lower().split(" ")
    
    if len(text1) > len(text2):
        text2,text1 = text1,text2  
    arr = [[0 for i in range(len(text2)+1)] for j in range(2)]
    
    b=1
    for i in range (1,len(text1)+1):    
        b= i & 1
        for j in range(1,len(text2)+1):
            if text1[i-1] == text2[j-1]:
                arr[1-b][j]= arr[b][j-1]+1
            else:
                arr[1-b][j] = max(arr[b][j],arr[1-b][j-1])
    
    return arr[1-b][-1]

In [ ]:
ans = {}
count = 0 
total_length_avalara_file = len(avalara_file)
per_done = 0
def cmp_avalara_unspsc():
  global ans,count,total_length_avalara_file,per_done
  for data in avalara_file:
    count+=1
    per_done = count * 100 /total_length_avalara_file
    levels = ["Segment Title","Family Title","Class Title","Commodity Title"]
    for level in levels:
      for match in (search_tree(data[3],level)):
        avalara_desc = data[1]
        unspsc_desc = match.data["title"]
        avalara_length = len(avalara_desc.split(" "))
        unspsc_length = len(unspsc_desc.split(" "))
        min_length = unspsc_length if (avalara_length > unspsc_length) else avalara_length
        thresold = 0.6
        match_perc = (longestCommonSubsequence(unspsc_desc,avalara_desc))/min_length
        if (match_perc >= thresold):
            if avalara_desc in ans:
              if level in ans[avalara_desc]:
                ans[avalara_desc][level].append({"unspsc_text" : unspsc_desc,"match_ratio" : match_perc })
              else:
                ans[avalara_desc][level] = []
                ans[avalara_desc][level].append({"unspsc_text" : unspsc_desc,"match_ratio" : match_perc })

            else:
              ans[avalara_desc] ={}
              ans[avalara_desc][level] = []
              ans[avalara_desc][level].append({"unspsc_text" : unspsc_desc,"match_ratio" : match_perc,"min_length" : min_length })


In [ ]:
cmp_avalara_unspsc()

KeyboardInterrupt: ignored

In [ ]:
len(ans)

300

In [ ]:
# Save the DataFrame to a file on the Colab backend.
import json
with open('ans.json', 'w') as f:
  f.write(json.dumps(ans) )


In [ ]:
from anytree.search import find 
list(map(lambda child_nodes: find(child_nodes,lambda node: node.name == "Land and Buildings and Structures and Thoroughfares"),root.children))